In [3]:
import mitsuba as mi
import drjit as dr
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random 
import imageio
import cv2
import numpy as np

In [4]:
scene_path = os.path.join('', 'scenes') 
results_spatio_path = os.path.join('','results_spatio') 
mi.set_variant('scalar_rgb')
texture_path = os.path.join(scene_path,'textures')

tokki = os.path.join(scene_path,'meshes/bunny.ply')


In [ ]:
cam1 = mi.load_dict({
'type': 'perspective',
'fov': 45, #24.6 mm
'to_world': mi.ScalarTransform4f.look_at(
    origin=[0.03, 0, -0.7],  # increase the z-coordinate to move the camera farther away
    target=[0.03, 1, -15],
    up=[0, 0, 1]
),
'film2': {
    'type': 'hdrfilm',
    'width': 360,
    'height': 360,
}
# 'distortion': [0, 0, 0, 0, 0]
})

In [ ]:
import cgi, os
files = os.listdir('dir/graycode_pattern/')

for f in files:
    s = "dir/graycode_pattern/"
    s = s + f
    scene1 = mi.load_dict({
        'type':'scene',
        'integrator': {'type': 'direct'},
        'light': {
            'type': 'projector',
            'irradiance': {
                'type': 'bitmap',
                'filename': s, 
            },
            'fov': 45,
            'to_world': mi.ScalarTransform4f.look_at(
                origin=[-0.03, 0.1, -0.7],
                target=[-0.03, -0.1, -15],
                up=[0, 0, 1]
            )
            },
        'tok':{
            'type': 'ply',
            'filename': tokki,
            'to_world': mi.ScalarTransform4f.translate([0, 0, -1]),
            'bsdf': {
                'type': 'diffuse',
                'reflectance': {'type': 'rgb', 'value': [0.1, 0.2, 0.3]},
            }
        }
    })

    image = mi.render(scene=scene1, sensor=cam1, spp=64)
    image = np.clip(image * 1000, 0, 1000).astype(np.uint8)
    st = "./results/resultfar_" + f
    # Save the image as a PNG file using imageio
    imageio.imwrite(st, image)


In [ ]:
help(cv2.Rodrigues)

Help on built-in function Rodrigues:

Rodrigues(...)
    Rodrigues(src[, dst[, jacobian]]) -> dst, jacobian
    .   @brief Converts a rotation matrix to a rotation vector or vice versa.
    .   
    .   @param src Input rotation vector (3x1 or 1x3) or rotation matrix (3x3).
    .   @param dst Output rotation matrix (3x3) or rotation vector (3x1 or 1x3), respectively.
    .   @param jacobian Optional output Jacobian matrix, 3x9 or 9x3, which is a matrix of partial
    .   derivatives of the output array components with respect to the input array components.
    .   
    .   \f[\begin{array}{l} \theta \leftarrow norm(r) \\ r  \leftarrow r/ \theta \\ R =  \cos(\theta) I + (1- \cos{\theta} ) r r^T +  \sin(\theta) \vecthreethree{0}{-r_z}{r_y}{r_z}{0}{-r_x}{-r_y}{r_x}{0} \end{array}\f]
    .   
    .   Inverse transformation can be also done easily, since
    .   
    .   \f[\sin ( \theta ) \vecthreethree{0}{-r_z}{r_y}{r_z}{0}{-r_x}{-r_y}{r_x}{0} = \frac{R - R^T}{2}\f]
    .   
    .   A rot

In [19]:
# Get the rotation matrix from the camera transformation matrix
cam_transform = mi.ScalarTransform4f.look_at(
    origin=[0.03, 0, -0.7],
    target=[0.03, 1, -15],
    up=[0, 0, 1]
)
cam_to_world = np.array(cam_transform.matrix)
cam_rot_vec, _ = cv2.Rodrigues(cam_to_world[:3, :3])
cam_rot_mat = np.array(cam_rot_vec)

# Define the proj_dist and proj_int matrices
proj_dist = np.zeros((4, 1))

proj =  mi.ScalarTransform4f.look_at(
            origin=[-0.03, 0.1, -0.7],
            target=[-0.03, -0.1, -15],
            up=[0, 0, 1]
        )

proj_int = np.array([[360/(2*np.tan(45*np.pi/360)) , 0, 360/2],
                     [0, 360/(2*np.tan(45*np.pi/360)), 360/2],
                     [0, 0, 1]])
# Compute the cam_proj_rmat matrix
# cam_proj_rmat = np.dot(proj_int, np.hstack((cam_rot_mat, -np.dot(cam_rot_mat, cam_to_world[:3, 3].reshape(-1,1)))))

proj_int = np.array([[434.55844123,   0.        , 180.        ],
       [  0.        , 434.55844123, 180.        ],
       [  0.        ,   0.        ,   1.        ]])

cam_int = np.array([[275.8, 0, 180], [0, 275.8, 180], [0, 0, 1]], dtype=np.float32)


cam_proj_rmat = np.dot(proj_int, cam_to_world[:3, :3])
cam_proj_rmat = np.dot(proj_int, cam_int)

cam_proj_rmat




array([[1.19851213e+05, 0.00000000e+00, 7.84005194e+04],
       [0.00000000e+00, 1.19851213e+05, 7.84005194e+04],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

SyntaxError: invalid syntax (1796576605.py, line 5)

In [18]:
proj_int = np.array([[360/(2*np.tan(45*np.pi/360)) , 0, 360/2],
                     [0, 360/(2*np.tan(45*np.pi/360)), 360/2],
                     [0, 0, 1]])
proj_int

array([[434.55844123,   0.        , 180.        ],
       [  0.        , 434.55844123, 180.        ],
       [  0.        ,   0.        ,   1.        ]])

In [8]:
# focal_length_px = 360 * (35 / 24.6)  # convert 35mm focal length to pixels
# fx = fy = focal_length_px
# cx = cy = 360 / 2  # assuming the principal point is at the center of the image
# s = 0  # assuming no skew
# cam_int = np.array([[fx, s, cx], [0, fy, cy], [0, 0, 1]])
# cam_int

array([[512.19512195,   0.        , 180.        ],
       [  0.        , 512.19512195, 180.        ],
       [  0.        ,   0.        ,   1.        ]])

In [20]:
u = 360
v = 360
point = np.array([u, v, 1])
cam_int = np.array([[275.8, 0, 180], [0, 275.8, 180], [0, 0, 1]], dtype=np.float32)


cam_proj_tvec = -np.linalg.inv(cam_proj_rmat) @ np.linalg.inv(cam_int) @ proj_int @ point
cam_proj_tvec

array([ 0.64941598,  0.64941598, -1.        ])